In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm

In [3]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

In [4]:
os.makedirs('data/adjusted_fields', exist_ok=True)

# OpenMRG adjustment

In [5]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc       
months = ['2015-06', '2015-07', '2015-08']                           

In [6]:
# methods parameter : default version
nnear = 12 
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.3}
diff_check_sel = 10
ratio_check_sel = (0.1,15)

In [7]:
# additive IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_idw.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:23<00:00, 30.96it/s]


month: 2015-07


100%|██████████| 744/744 [00:21<00:00, 33.97it/s]


month: 2015-08


100%|██████████| 744/744 [00:23<00:00, 31.81it/s]


In [8]:
# multiplicative IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_idw.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:25<00:00, 28.17it/s]


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.55it/s]


month: 2015-08


100%|██████████| 744/744 [00:22<00:00, 32.51it/s]


In [9]:
# additive POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:34<00:00, 20.83it/s]


month: 2015-07


100%|██████████| 744/744 [00:41<00:00, 18.00it/s]


month: 2015-08


100%|██████████| 744/744 [00:34<00:00, 21.67it/s]


In [10]:
# multiplicative POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:28<00:00, 25.51it/s]


month: 2015-07


100%|██████████| 744/744 [00:31<00:00, 23.54it/s]


month: 2015-08


100%|██████████| 744/744 [00:24<00:00, 30.32it/s]


In [11]:
# additive BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_b_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:28<00:00, 25.23it/s]


month: 2015-07


100%|██████████| 744/744 [00:33<00:00, 21.90it/s]


month: 2015-08


100%|██████████| 744/744 [00:29<00:00, 25.39it/s]


In [12]:
# multiplicative BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_b_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:24<00:00, 29.28it/s]


month: 2015-07


100%|██████████| 744/744 [00:29<00:00, 25.50it/s]


month: 2015-08


100%|██████████| 744/744 [00:23<00:00, 32.14it/s]


In [13]:
# KED point
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_p.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:28<00:00, 25.15it/s]


month: 2015-07


100%|██████████| 744/744 [00:33<00:00, 22.35it/s]


month: 2015-08


100%|██████████| 744/744 [00:28<00:00, 25.91it/s]


In [14]:
# KED block
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_b.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:27<00:00, 26.04it/s]


month: 2015-07


100%|██████████| 744/744 [00:33<00:00, 22.10it/s]


month: 2015-08


100%|██████████| 744/744 [00:35<00:00, 20.81it/s]


# OpenRainER adjustment

In [15]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc      
months = ['2022-06', '2022-07', '2022-08']

In [16]:
# methods parameter : default version
nnear = 12 
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.3}
diff_check_sel = 10
ratio_check_sel = (0.1,15)

In [17]:
# additive IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_idw.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [01:24<00:00,  8.51it/s]


month: 2022-07


100%|██████████| 744/744 [01:21<00:00,  9.13it/s]


month: 2022-08


100%|██████████| 744/744 [01:36<00:00,  7.70it/s]


In [18]:
# multiplicative IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_idw.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [01:17<00:00,  9.29it/s]


month: 2022-07


100%|██████████| 744/744 [01:23<00:00,  8.94it/s]


month: 2022-08


100%|██████████| 744/744 [01:33<00:00,  7.93it/s]


In [19]:
# additive POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [09:54<00:00,  1.21it/s]


month: 2022-07


100%|██████████| 744/744 [02:28<00:00,  5.01it/s]


month: 2022-08


100%|██████████| 744/744 [07:11<00:00,  1.72it/s]


In [20]:
# multiplicative POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [05:09<00:00,  2.32it/s]


month: 2022-07


100%|██████████| 744/744 [01:34<00:00,  7.90it/s]


month: 2022-08


100%|██████████| 744/744 [04:56<00:00,  2.51it/s]


In [21]:
# additive BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_b_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [09:28<00:00,  1.27it/s]


month: 2022-07


100%|██████████| 744/744 [02:35<00:00,  4.79it/s]


month: 2022-08


100%|██████████| 744/744 [07:09<00:00,  1.73it/s]


In [22]:
# multiplicative BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field  
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_b_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [04:52<00:00,  2.45it/s]


month: 2022-07


100%|██████████| 744/744 [01:28<00:00,  8.38it/s] 


month: 2022-08


100%|██████████| 744/744 [04:38<00:00,  2.67it/s]


In [23]:
# KED point
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_p.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:51<00:00,  1.35it/s]


month: 2022-07


100%|██████████| 744/744 [02:13<00:00,  5.57it/s]


month: 2022-08


100%|██████████| 744/744 [06:36<00:00,  1.88it/s]


In [24]:
# KED block
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_b.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:50<00:00,  1.36it/s]


month: 2022-07


100%|██████████| 744/744 [02:10<00:00,  5.69it/s]


month: 2022-08


100%|██████████| 744/744 [06:38<00:00,  1.87it/s]


In [25]:
data = xr.open_dataset('data/adjusted_fields/OpenRainER_ked_b.nc')